In [1]:
# import sys
# sys.path
# sys.path.insert(0, '/Users/zhengzaezhou/scikit-learn')

In [2]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from scipy import stats
from tqdm import tqdm

from sklearn.utils import shuffle


/Users/zhengzezhou/scikit-learn/sklearn/utils/__init__.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.9 it will stop working
  from collections import Sequence
/Users/zhengzezhou/scikit-learn/sklearn/model_selection/_split.py:18: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.9 it will stop working
  from collections import Iterable
/Users/zhengzezhou/scikit-learn/sklearn/model_selection/_search.py:16: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.9 it will stop working
  from collections import Mapping, namedtuple, defaultdict, Sequence


In [3]:
def predict_var(self, X, method = "corrected"):

    predict_all = np.zeros((X.shape[0], self.n_estimators))
    for t_idx in range(self.n_estimators):
        predict_all[:, t_idx] = self.estimators_[t_idx].predict(X)
    pred = np.mean(predict_all, axis = 1)

    inbag_times_ = self.inbag_times_

    m = X.shape[0]

    if method == "BM":
        cond_exp_full = np.zeros((self.n_samples_, m))

        for i in range(self.n_samples_):
            # cond_exp_full[i, :] = np.mean(predict_all[:, inbag_times_[i, :] >= 1], axis=1)
            cond_exp_full[i, :] = np.average(predict_all, weights = inbag_times_[i, :], axis = 1)

        zeta1_full = np.zeros(m)
        zetan_full = np.zeros(m)
        variance = np.zeros(m)
        for i in range(m):
            zeta1_full[i] = np.var(cond_exp_full[:, i])
            zetan_full[i] = np.var(predict_all[i, :])
        variance = zeta1_full * (self.n_subsamples_ ** 2) / self.n_samples_ + zetan_full / self.n_estimators
        
        return [float(variance), float(zeta1_full), float(zetan_full)]
        
    elif method == "IJ":
        
        f_centered = predict_all - np.mean(predict_all, axis=1).reshape(m, 1)
        i_centered = inbag_times_ - np.mean(inbag_times_, axis=1).reshape(self.n_samples_, 1)
        corr = np.dot(f_centered, i_centered.T) / self.n_estimators
        cov = np.dot(corr, corr.T)
        zetan_full = np.cov(predict_all)
        covariance = cov + zetan_full / self.n_estimators

        return [float(np.diagonal(covariance)), float(cov), float(zetan_full)]
    
    elif method == "corrected":
        
        cond_exp_full = np.zeros((self.n_samples_, m))

        for i in range(self.n_samples_):
            # cond_exp_full[i, :] = np.mean(predict_all[:, inbag_times_[i, :] >= 1], axis=1)
            cond_exp_full[i, :] = np.average(predict_all, weights = inbag_times_[i, :], axis = 1)

        inbag_times_ = inbag_times_[np.sum(inbag_times_, axis = 1) > 0, ]

        nk = np.sum(inbag_times_, axis = 1)
        K = len(nk)
        C = np.sum(nk)     

        SSr = np.dot(((cond_exp_full - pred) ** 2).T, nk)

        SSe = [0] * m

        for i in range(K):
            SSei = np.sum((predict_all[:, inbag_times_[i, :]>=1].T - cond_exp_full[i, :]) ** 2, axis = 0)
            SSe += SSei

        sigma_e_squared = SSe / (C - K)

        sigma_M_squared = (SSr - (K - 1) * sigma_e_squared) / (C - np.sum(nk ** 2) / C)
        
        sigma_M_squared_without = SSr / C
        
        return  [(self.n_subsamples_ ** 2) / self.n_samples_ * float(sigma_M_squared), (self.n_subsamples_ ** 2) / self.n_samples_ * float(sigma_M_squared_without)]
        

In [4]:
df = pd.read_csv("CASP.csv")

In [5]:
df.head()

,RMSD,F1,F2,F3,F4,F5,F6,F7,F8,F9
0,17.284,13558.30,4305.35,0.31754,162.1730,1.872791e+06,215.3590,4287.87,102,27.0302
1,6.021,6191.96,1623.16,0.26213,53.3894,8.034467e+05,87.2024,3328.91,39,38.5468
2,9.275,7725.98,1726.28,0.22343,67.2887,1.075648e+06,81.7913,2981.04,29,38.8119
3,15.851,8424.58,2368.25,0.28111,67.8325,1.210472e+06,109.4390,3248.22,70,39.0651
4,7.962,7460.84,1736.94,0.23280,52.4123,1.021020e+06,94.5234,2814.42,41,39.9147


In [6]:
df = shuffle(df)
df.head()

,RMSD,F1,F2,F3,F4,F5,F6,F7,F8,F9
44265,11.957,4562.42,1793.38,0.39307,41.7351,6.171690e+05,74.8099,1495.88,28,43.5470
33985,6.588,15100.40,4166.17,0.27589,192.1600,2.066682e+06,252.5900,5200.19,87,26.0274
21729,13.678,12281.30,3547.89,0.28888,130.0640,1.707463e+06,161.3940,4252.15,32,31.0761
33136,1.497,8648.83,2301.76,0.26613,88.1917,1.230148e+06,119.5850,3760.52,5,34.6373
1408,16.700,7685.96,3209.73,0.41760,72.4252,1.094516e+06,111.4950,3203.68,31,36.0927


In [7]:
df.describe()

,RMSD,F1,F2,F3,F4,F5,F6,F7,F8,F9
count,45730.000000,45730.000000,45730.000000,45730.000000,45730.000000,4.573000e+04,45730.000000,45730.000000,45730.000000,45730.000000
mean,7.748528,9871.596995,3017.367175,0.302392,103.492433,1.368299e+06,145.638061,3989.755990,69.975071,34.523664
std,6.118312,4058.138034,1464.324663,0.062886,55.424985,5.640367e+05,69.999230,1993.574575,56.493443,5.979755
min,0.000000,2392.050000,403.500000,0.092500,10.310100,3.194902e+05,31.970400,0.000000,0.000000,15.228000
25%,2.305000,6936.680000,1979.045000,0.258740,63.563900,9.535912e+05,94.757500,3165.322500,31.000000,30.424725
50%,5.030000,8898.805000,2668.155000,0.300150,87.740800,1.237219e+06,126.176000,3840.170000,54.000000,35.299300
75%,13.379000,12126.150000,3786.410000,0.342890,133.646750,1.690920e+06,181.468500,4644.192500,91.000000,38.870800
max,20.999000,40034.900000,15312.000000,0.577690,369.317000,5.472011e+06,598.408000,105948.170000,350.000000,55.300900


In [8]:
df_train = df[:45000]
df_test = df[45000:]

## k = 100

In [9]:
result_100 = {500:  {"normal": [], "vr_uc": [], "cc_uc": []},
              1000: {"normal": [], "vr_uc": [], "cc_uc": []},
              2500: {"normal": [], "vr_uc": [], "cc_uc": []},
              5000: {"normal": [], "vr_uc": [], "cc_uc": []}}

rep = 45
incre = 1000
n_sub = 100

for _ in tqdm(range(20)):
    
    test = df_test.iloc[np.random.choice(730)].values[1:].reshape(1, -1)

    # print("========================= B = 500 =========================")

    pred = []
    var_uc = []
    var_c = []

    for r in range(rep):

        data = df_train[incre * r: (incre * r + incre)]

        X = data[["F1", "F2", "F3", "F4", "F5", "F6", "F7", "F8", "F9"]].values
        y = data[["RMSD"]].values.ravel()

        model = RandomForestRegressor(n_estimators = 500)
        model.fit(X, y, n_subsamples = n_sub, replace = False)
        pred.append(model.predict(test)[0])
#         var_c.append(predict_var(model, test)[0])
        var_uc.append(predict_var(model, test, method = "IJ")[0])

    target = np.mean(pred)
#     count_c = 0
#     for r in range(rep):
#         lower = pred[r] - 1.96 * np.sqrt(var_c[r])
#         upper = pred[r] + 1.96 * np.sqrt(var_c[r])
#         if target >= lower and target <= upper:
#             count_c += 1
    count_uc = 0
    for r in range(rep):
        lower = pred[r] - 1.96 * np.sqrt(var_uc[r])
        upper = pred[r] + 1.96 * np.sqrt(var_uc[r])
        if target >= lower and target <= upper:
            count_uc += 1        
    
    result_100[500]["normal"].append(stats.normaltest(pred))
#     result_100[500]["vr_c"].append(np.mean(var_c) / np.var(pred))
    result_100[500]["vr_uc"].append(np.mean(var_uc) / np.var(pred))
#     result_100[500]["cc_c"].append(count_c / rep)
    result_100[500]["cc_uc"].append(count_uc / rep)


    # print("========================= B = 1000 =========================")
    
    pred = []
    var_uc = []
    var_c = []

    for r in range(rep):

        data = df_train[incre * r: (incre * r + incre)]

        X = data[["F1", "F2", "F3", "F4", "F5", "F6", "F7", "F8", "F9"]].values
        y = data[["RMSD"]].values.ravel()

        model = RandomForestRegressor(n_estimators = 1000)
        model.fit(X, y, n_subsamples = n_sub, replace = False)
        pred.append(model.predict(test)[0])
#         var_c.append(predict_var(model, test)[0])
        var_uc.append(predict_var(model, test, method = "IJ")[0])

    target = np.mean(pred)
#     count_c = 0
#     for r in range(rep):
#         lower = pred[r] - 1.96 * np.sqrt(var_c[r])
#         upper = pred[r] + 1.96 * np.sqrt(var_c[r])
#         if target >= lower and target <= upper:
#             count_c += 1
    count_uc = 0
    for r in range(rep):
        lower = pred[r] - 1.96 * np.sqrt(var_uc[r])
        upper = pred[r] + 1.96 * np.sqrt(var_uc[r])
        if target >= lower and target <= upper:
            count_uc += 1        
    
    result_100[1000]["normal"].append(stats.normaltest(pred))
#     result_100[500]["vr_c"].append(np.mean(var_c) / np.var(pred))
    result_100[1000]["vr_uc"].append(np.mean(var_uc) / np.var(pred))
#     result_100[500]["cc_c"].append(count_c / rep)
    result_100[1000]["cc_uc"].append(count_uc / rep)



    # print("========================= B = 2500 =========================")
    pred = []
    var_uc = []
    var_c = []

    for r in range(rep):

        data = df_train[incre * r: (incre * r + incre)]

        X = data[["F1", "F2", "F3", "F4", "F5", "F6", "F7", "F8", "F9"]].values
        y = data[["RMSD"]].values.ravel()

        model = RandomForestRegressor(n_estimators = 2500)
        model.fit(X, y, n_subsamples = n_sub, replace = False)
        pred.append(model.predict(test)[0])
#         var_c.append(predict_var(model, test)[0])
        var_uc.append(predict_var(model, test, method = "IJ")[0])

    target = np.mean(pred)
#     count_c = 0
#     for r in range(rep):
#         lower = pred[r] - 1.96 * np.sqrt(var_c[r])
#         upper = pred[r] + 1.96 * np.sqrt(var_c[r])
#         if target >= lower and target <= upper:
#             count_c += 1
    count_uc = 0
    for r in range(rep):
        lower = pred[r] - 1.96 * np.sqrt(var_uc[r])
        upper = pred[r] + 1.96 * np.sqrt(var_uc[r])
        if target >= lower and target <= upper:
            count_uc += 1        
    
    result_100[2500]["normal"].append(stats.normaltest(pred))
#     result_100[500]["vr_c"].append(np.mean(var_c) / np.var(pred))
    result_100[2500]["vr_uc"].append(np.mean(var_uc) / np.var(pred))
#     result_100[500]["cc_c"].append(count_c / rep)
    result_100[2500]["cc_uc"].append(count_uc / rep)


    # print("========================= B = 5000 =========================")
    pred = []
    var_uc = []
    var_c = []

    for r in range(rep):

        data = df_train[incre * r: (incre * r + incre)]

        X = data[["F1", "F2", "F3", "F4", "F5", "F6", "F7", "F8", "F9"]].values
        y = data[["RMSD"]].values.ravel()

        model = RandomForestRegressor(n_estimators = 5000)
        model.fit(X, y, n_subsamples = n_sub, replace = False)
        pred.append(model.predict(test)[0])
#         var_c.append(predict_var(model, test)[0])
        var_uc.append(predict_var(model, test, method = "IJ")[0])

    target = np.mean(pred)
#     count_c = 0
#     for r in range(rep):
#         lower = pred[r] - 1.96 * np.sqrt(var_c[r])
#         upper = pred[r] + 1.96 * np.sqrt(var_c[r])
#         if target >= lower and target <= upper:
#             count_c += 1
    count_uc = 0
    for r in range(rep):
        lower = pred[r] - 1.96 * np.sqrt(var_uc[r])
        upper = pred[r] + 1.96 * np.sqrt(var_uc[r])
        if target >= lower and target <= upper:
            count_uc += 1        
    
    result_100[5000]["normal"].append(stats.normaltest(pred))
#     result_100[500]["vr_c"].append(np.mean(var_c) / np.var(pred))
    result_100[5000]["vr_uc"].append(np.mean(var_uc) / np.var(pred))
#     result_100[500]["cc_c"].append(count_c / rep)
    result_100[5000]["cc_uc"].append(count_uc / rep)



100%|██████████| 20/20 [3:39:55<00:00, 659.78s/it]  


In [10]:
result_100

{500: {'normal': [NormaltestResult(statistic=4.4590371394518815, pvalue=0.10758021004059116),
   NormaltestResult(statistic=0.523149291958275, pvalue=0.7698384079377711),
   NormaltestResult(statistic=1.9497558919268112, pvalue=0.37723839422214883),
   NormaltestResult(statistic=0.7939596691431413, pvalue=0.6723475836730333),
   NormaltestResult(statistic=3.781929849684468, pvalue=0.150926106202414),
   NormaltestResult(statistic=1.3774979155829687, pvalue=0.5022039545729817),
   NormaltestResult(statistic=1.0125682649397625, pvalue=0.602731091698159),
   NormaltestResult(statistic=12.71377565629451, pvalue=0.0017347571819446785),
   NormaltestResult(statistic=2.4251174578090544, pvalue=0.29743524876246324),
   NormaltestResult(statistic=0.012147980388537882, pvalue=0.993944419192501),
   NormaltestResult(statistic=8.778184128665039, pvalue=0.012411993397605717),
   NormaltestResult(statistic=1.3020586681842343, pvalue=0.5215086937117179),
   NormaltestResult(statistic=0.33757028705366

In [12]:
result = result_100

print("---------- B_n = 500 ----------")
B_n = 500
print("normality rejection rate: " + str(np.sum(np.array([i.pvalue for i in result[B_n]["normal"]]) < 0.05) / 20))
# print("mean variance ratio (corrected): " + str(np.mean(result[B_n]["vr_c"])))
# print("std variance ratio (corrected): " + str(np.std(result[B_n]["vr_c"])))
print("mean variance ratio (uncorrected): " + str(np.mean(result[B_n]["vr_uc"])))
print("std variance ratio (uncorrected): " + str(np.std(result[B_n]["vr_uc"])))
# print("mean coverage prob (corrected): " + str(np.mean(result[B_n]["cc_c"])))
# print("std coverage prob (corrected): " + str(np.std(result[B_n]["cc_c"])))
print("mean coverage prob (uncorrected): " + str(np.mean(result[B_n]["cc_uc"])))
print("std coverage prob (uncorrected): " + str(np.std(result[B_n]["cc_uc"])))

print("")
print("---------- B_n = 1000 ----------")
B_n = 1000
print("normality rejection rate: " + str(np.sum(np.array([i.pvalue for i in result[B_n]["normal"]]) < 0.05) / 20))
# print("mean variance ratio (corrected): " + str(np.mean(result[B_n]["vr_c"])))
# print("std variance ratio (corrected): " + str(np.std(result[B_n]["vr_c"])))
print("mean variance ratio (uncorrected): " + str(np.mean(result[B_n]["vr_uc"])))
print("std variance ratio (uncorrected): " + str(np.std(result[B_n]["vr_uc"])))
# print("mean coverage prob (corrected): " + str(np.mean(result[B_n]["cc_c"])))
# print("std coverage prob (corrected): " + str(np.std(result[B_n]["cc_c"])))
print("mean coverage prob (uncorrected): " + str(np.mean(result[B_n]["cc_uc"])))
print("std coverage prob (uncorrected): " + str(np.std(result[B_n]["cc_uc"])))

print("")
print("---------- B_n = 2500 ----------")
B_n = 2500
print("normality rejection rate: " + str(np.sum(np.array([i.pvalue for i in result[B_n]["normal"]]) < 0.05) / 20))
# print("mean variance ratio (corrected): " + str(np.mean(result[B_n]["vr_c"])))
# print("std variance ratio (corrected): " + str(np.std(result[B_n]["vr_c"])))
print("mean variance ratio (uncorrected): " + str(np.mean(result[B_n]["vr_uc"])))
print("std variance ratio (uncorrected): " + str(np.std(result[B_n]["vr_uc"])))
# print("mean coverage prob (corrected): " + str(np.mean(result[B_n]["cc_c"])))
# print("std coverage prob (corrected): " + str(np.std(result[B_n]["cc_c"])))
print("mean coverage prob (uncorrected): " + str(np.mean(result[B_n]["cc_uc"])))
print("std coverage prob (uncorrected): " + str(np.std(result[B_n]["cc_uc"])))

print("")
print("---------- B_n = 5000 ----------")
B_n = 5000
print("normality rejection rate: " + str(np.sum(np.array([i.pvalue for i in result[B_n]["normal"]]) < 0.05) / 20))
# print("mean variance ratio (corrected): " + str(np.mean(result[B_n]["vr_c"])))
# print("std variance ratio (corrected): " + str(np.std(result[B_n]["vr_c"])))
print("mean variance ratio (uncorrected): " + str(np.mean(result[B_n]["vr_uc"])))
print("std variance ratio (uncorrected): " + str(np.std(result[B_n]["vr_uc"])))
# print("mean coverage prob (corrected): " + str(np.mean(result[B_n]["cc_c"])))
# print("std coverage prob (corrected): " + str(np.std(result[B_n]["cc_c"])))
print("mean coverage prob (uncorrected): " + str(np.mean(result[B_n]["cc_uc"])))
print("std coverage prob (uncorrected): " + str(np.std(result[B_n]["cc_uc"])))

---------- B_n = 500 ----------
normality rejection rate: 0.1
mean variance ratio (uncorrected): 10.774247085034428
std variance ratio (uncorrected): 2.6463787131760736
mean coverage prob (uncorrected): 1.0
std coverage prob (uncorrected): 0.0

---------- B_n = 1000 ----------
normality rejection rate: 0.15
mean variance ratio (uncorrected): 6.032034589675479
std variance ratio (uncorrected): 1.4904726103330388
mean coverage prob (uncorrected): 1.0
std coverage prob (uncorrected): 0.0

---------- B_n = 2500 ----------
normality rejection rate: 0.1
mean variance ratio (uncorrected): 3.0717749334803504
std variance ratio (uncorrected): 0.7723846501299466
mean coverage prob (uncorrected): 0.9933333333333334
std coverage prob (uncorrected): 0.014229164972072987

---------- B_n = 5000 ----------
normality rejection rate: 0.2
mean variance ratio (uncorrected): 1.9959287621158133
std variance ratio (uncorrected): 0.45627650447615553
mean coverage prob (uncorrected): 0.9822222222222223
std cov

## k = 250

In [13]:
result_250 = {500:  {"normal": [], "vr_uc": [], "cc_uc": []},
              1000: {"normal": [], "vr_uc": [], "cc_uc": []},
              2500: {"normal": [], "vr_uc": [], "cc_uc": []},
              5000: {"normal": [], "vr_uc": [], "cc_uc": []}}

rep = 45
incre = 1000
n_sub = 250

for _ in tqdm(range(20)):
    
    test = df_test.iloc[np.random.choice(730)].values[1:].reshape(1, -1)

    # print("========================= B = 500 =========================")

    pred = []
    var_uc = []
    var_c = []

    for r in range(rep):

        data = df_train[incre * r: (incre * r + incre)]

        X = data[["F1", "F2", "F3", "F4", "F5", "F6", "F7", "F8", "F9"]].values
        y = data[["RMSD"]].values.ravel()

        model = RandomForestRegressor(n_estimators = 500)
        model.fit(X, y, n_subsamples = n_sub, replace = False)
        pred.append(model.predict(test)[0])
#         var_c.append(predict_var(model, test)[0])
        var_uc.append(predict_var(model, test, method = "IJ")[0])

    target = np.mean(pred)
#     count_c = 0
#     for r in range(rep):
#         lower = pred[r] - 1.96 * np.sqrt(var_c[r])
#         upper = pred[r] + 1.96 * np.sqrt(var_c[r])
#         if target >= lower and target <= upper:
#             count_c += 1
    count_uc = 0
    for r in range(rep):
        lower = pred[r] - 1.96 * np.sqrt(var_uc[r])
        upper = pred[r] + 1.96 * np.sqrt(var_uc[r])
        if target >= lower and target <= upper:
            count_uc += 1        
    
    result_250[500]["normal"].append(stats.normaltest(pred))
#     result_100[500]["vr_c"].append(np.mean(var_c) / np.var(pred))
    result_250[500]["vr_uc"].append(np.mean(var_uc) / np.var(pred))
#     result_100[500]["cc_c"].append(count_c / rep)
    result_250[500]["cc_uc"].append(count_uc / rep)


    # print("========================= B = 1000 =========================")
    
    pred = []
    var_uc = []
    var_c = []

    for r in range(rep):

        data = df_train[incre * r: (incre * r + incre)]

        X = data[["F1", "F2", "F3", "F4", "F5", "F6", "F7", "F8", "F9"]].values
        y = data[["RMSD"]].values.ravel()

        model = RandomForestRegressor(n_estimators = 1000)
        model.fit(X, y, n_subsamples = n_sub, replace = False)
        pred.append(model.predict(test)[0])
#         var_c.append(predict_var(model, test)[0])
        var_uc.append(predict_var(model, test, method = "IJ")[0])

    target = np.mean(pred)
#     count_c = 0
#     for r in range(rep):
#         lower = pred[r] - 1.96 * np.sqrt(var_c[r])
#         upper = pred[r] + 1.96 * np.sqrt(var_c[r])
#         if target >= lower and target <= upper:
#             count_c += 1
    count_uc = 0
    for r in range(rep):
        lower = pred[r] - 1.96 * np.sqrt(var_uc[r])
        upper = pred[r] + 1.96 * np.sqrt(var_uc[r])
        if target >= lower and target <= upper:
            count_uc += 1        
    
    result_250[1000]["normal"].append(stats.normaltest(pred))
#     result_100[500]["vr_c"].append(np.mean(var_c) / np.var(pred))
    result_250[1000]["vr_uc"].append(np.mean(var_uc) / np.var(pred))
#     result_100[500]["cc_c"].append(count_c / rep)
    result_250[1000]["cc_uc"].append(count_uc / rep)



    # print("========================= B = 2500 =========================")
    pred = []
    var_uc = []
    var_c = []

    for r in range(rep):

        data = df_train[incre * r: (incre * r + incre)]

        X = data[["F1", "F2", "F3", "F4", "F5", "F6", "F7", "F8", "F9"]].values
        y = data[["RMSD"]].values.ravel()

        model = RandomForestRegressor(n_estimators = 2500)
        model.fit(X, y, n_subsamples = n_sub, replace = False)
        pred.append(model.predict(test)[0])
#         var_c.append(predict_var(model, test)[0])
        var_uc.append(predict_var(model, test, method = "IJ")[0])

    target = np.mean(pred)
#     count_c = 0
#     for r in range(rep):
#         lower = pred[r] - 1.96 * np.sqrt(var_c[r])
#         upper = pred[r] + 1.96 * np.sqrt(var_c[r])
#         if target >= lower and target <= upper:
#             count_c += 1
    count_uc = 0
    for r in range(rep):
        lower = pred[r] - 1.96 * np.sqrt(var_uc[r])
        upper = pred[r] + 1.96 * np.sqrt(var_uc[r])
        if target >= lower and target <= upper:
            count_uc += 1        
    
    result_250[2500]["normal"].append(stats.normaltest(pred))
#     result_100[500]["vr_c"].append(np.mean(var_c) / np.var(pred))
    result_250[2500]["vr_uc"].append(np.mean(var_uc) / np.var(pred))
#     result_100[500]["cc_c"].append(count_c / rep)
    result_250[2500]["cc_uc"].append(count_uc / rep)


    # print("========================= B = 5000 =========================")
    pred = []
    var_uc = []
    var_c = []

    for r in range(rep):

        data = df_train[incre * r: (incre * r + incre)]

        X = data[["F1", "F2", "F3", "F4", "F5", "F6", "F7", "F8", "F9"]].values
        y = data[["RMSD"]].values.ravel()

        model = RandomForestRegressor(n_estimators = 5000)
        model.fit(X, y, n_subsamples = n_sub, replace = False)
        pred.append(model.predict(test)[0])
#         var_c.append(predict_var(model, test)[0])
        var_uc.append(predict_var(model, test, method = "IJ")[0])

    target = np.mean(pred)
#     count_c = 0
#     for r in range(rep):
#         lower = pred[r] - 1.96 * np.sqrt(var_c[r])
#         upper = pred[r] + 1.96 * np.sqrt(var_c[r])
#         if target >= lower and target <= upper:
#             count_c += 1
    count_uc = 0
    for r in range(rep):
        lower = pred[r] - 1.96 * np.sqrt(var_uc[r])
        upper = pred[r] + 1.96 * np.sqrt(var_uc[r])
        if target >= lower and target <= upper:
            count_uc += 1        
    
    result_250[5000]["normal"].append(stats.normaltest(pred))
#     result_100[500]["vr_c"].append(np.mean(var_c) / np.var(pred))
    result_250[5000]["vr_uc"].append(np.mean(var_uc) / np.var(pred))
#     result_100[500]["cc_c"].append(count_c / rep)
    result_250[5000]["cc_uc"].append(count_uc / rep)



100%|██████████| 20/20 [5:17:12<00:00, 951.61s/it]  


In [14]:
result_250

{500: {'normal': [NormaltestResult(statistic=5.794444186058774, pvalue=0.05517628194035611),
   NormaltestResult(statistic=0.6611483998122739, pvalue=0.7185110459855553),
   NormaltestResult(statistic=0.33118811210235266, pvalue=0.8473901572845695),
   NormaltestResult(statistic=1.512454145029618, pvalue=0.46943423132797135),
   NormaltestResult(statistic=2.8614403944381728, pvalue=0.2391366346714142),
   NormaltestResult(statistic=1.3631566324292748, pvalue=0.505818021232334),
   NormaltestResult(statistic=8.514192452635239, pvalue=0.014163369976840397),
   NormaltestResult(statistic=11.156259599942432, pvalue=0.0037796275726022796),
   NormaltestResult(statistic=9.948333781963523, pvalue=0.006914276887051657),
   NormaltestResult(statistic=1.9704683455585976, pvalue=0.37335178787331375),
   NormaltestResult(statistic=2.7530423352917883, pvalue=0.2524552767729537),
   NormaltestResult(statistic=1.7838158506995623, pvalue=0.4098729991867681),
   NormaltestResult(statistic=1.38400134623

In [15]:
result = result_250

print("---------- B_n = 500 ----------")
B_n = 500
print("normality rejection rate: " + str(np.sum(np.array([i.pvalue for i in result[B_n]["normal"]]) < 0.05) / 20))
# print("mean variance ratio (corrected): " + str(np.mean(result[B_n]["vr_c"])))
# print("std variance ratio (corrected): " + str(np.std(result[B_n]["vr_c"])))
print("mean variance ratio (uncorrected): " + str(np.mean(result[B_n]["vr_uc"])))
print("std variance ratio (uncorrected): " + str(np.std(result[B_n]["vr_uc"])))
# print("mean coverage prob (corrected): " + str(np.mean(result[B_n]["cc_c"])))
# print("std coverage prob (corrected): " + str(np.std(result[B_n]["cc_c"])))
print("mean coverage prob (uncorrected): " + str(np.mean(result[B_n]["cc_uc"])))
print("std coverage prob (uncorrected): " + str(np.std(result[B_n]["cc_uc"])))

print("")
print("---------- B_n = 1000 ----------")
B_n = 1000
print("normality rejection rate: " + str(np.sum(np.array([i.pvalue for i in result[B_n]["normal"]]) < 0.05) / 20))
# print("mean variance ratio (corrected): " + str(np.mean(result[B_n]["vr_c"])))
# print("std variance ratio (corrected): " + str(np.std(result[B_n]["vr_c"])))
print("mean variance ratio (uncorrected): " + str(np.mean(result[B_n]["vr_uc"])))
print("std variance ratio (uncorrected): " + str(np.std(result[B_n]["vr_uc"])))
# print("mean coverage prob (corrected): " + str(np.mean(result[B_n]["cc_c"])))
# print("std coverage prob (corrected): " + str(np.std(result[B_n]["cc_c"])))
print("mean coverage prob (uncorrected): " + str(np.mean(result[B_n]["cc_uc"])))
print("std coverage prob (uncorrected): " + str(np.std(result[B_n]["cc_uc"])))

print("")
print("---------- B_n = 2500 ----------")
B_n = 2500
print("normality rejection rate: " + str(np.sum(np.array([i.pvalue for i in result[B_n]["normal"]]) < 0.05) / 20))
# print("mean variance ratio (corrected): " + str(np.mean(result[B_n]["vr_c"])))
# print("std variance ratio (corrected): " + str(np.std(result[B_n]["vr_c"])))
print("mean variance ratio (uncorrected): " + str(np.mean(result[B_n]["vr_uc"])))
print("std variance ratio (uncorrected): " + str(np.std(result[B_n]["vr_uc"])))
# print("mean coverage prob (corrected): " + str(np.mean(result[B_n]["cc_c"])))
# print("std coverage prob (corrected): " + str(np.std(result[B_n]["cc_c"])))
print("mean coverage prob (uncorrected): " + str(np.mean(result[B_n]["cc_uc"])))
print("std coverage prob (uncorrected): " + str(np.std(result[B_n]["cc_uc"])))

print("")
print("---------- B_n = 5000 ----------")
B_n = 5000
print("normality rejection rate: " + str(np.sum(np.array([i.pvalue for i in result[B_n]["normal"]]) < 0.05) / 20))
# print("mean variance ratio (corrected): " + str(np.mean(result[B_n]["vr_c"])))
# print("std variance ratio (corrected): " + str(np.std(result[B_n]["vr_c"])))
print("mean variance ratio (uncorrected): " + str(np.mean(result[B_n]["vr_uc"])))
print("std variance ratio (uncorrected): " + str(np.std(result[B_n]["vr_uc"])))
# print("mean coverage prob (corrected): " + str(np.mean(result[B_n]["cc_c"])))
# print("std coverage prob (corrected): " + str(np.std(result[B_n]["cc_c"])))
print("mean coverage prob (uncorrected): " + str(np.mean(result[B_n]["cc_uc"])))
print("std coverage prob (uncorrected): " + str(np.std(result[B_n]["cc_uc"])))

---------- B_n = 500 ----------
normality rejection rate: 0.15
mean variance ratio (uncorrected): 13.199427052744051
std variance ratio (uncorrected): 5.703848395622799
mean coverage prob (uncorrected): 1.0
std coverage prob (uncorrected): 0.0

---------- B_n = 1000 ----------
normality rejection rate: 0.15
mean variance ratio (uncorrected): 6.951695206211831
std variance ratio (uncorrected): 2.4605410216620642
mean coverage prob (uncorrected): 1.0
std coverage prob (uncorrected): 0.0

---------- B_n = 2500 ----------
normality rejection rate: 0.2
mean variance ratio (uncorrected): 3.292750996467423
std variance ratio (uncorrected): 1.2044988898458577
mean coverage prob (uncorrected): 0.9966666666666667
std coverage prob (uncorrected): 0.007934920476158734

---------- B_n = 5000 ----------
normality rejection rate: 0.15
mean variance ratio (uncorrected): 2.0544212405726823
std variance ratio (uncorrected): 0.7215384219051323
mean coverage prob (uncorrected): 0.9766666666666668
std cove

## k = 500

In [16]:
result_500 = {500:  {"normal": [], "vr_uc": [], "cc_uc": []},
              1000: {"normal": [], "vr_uc": [], "cc_uc": []},
              2500: {"normal": [], "vr_uc": [], "cc_uc": []},
              5000: {"normal": [], "vr_uc": [], "cc_uc": []}}

rep = 45
incre = 1000
n_sub = 500

for _ in tqdm(range(20)):
    
    test = df_test.iloc[np.random.choice(730)].values[1:].reshape(1, -1)

    # print("========================= B = 500 =========================")

    pred = []
    var_uc = []
    var_c = []

    for r in range(rep):

        data = df_train[incre * r: (incre * r + incre)]

        X = data[["F1", "F2", "F3", "F4", "F5", "F6", "F7", "F8", "F9"]].values
        y = data[["RMSD"]].values.ravel()

        model = RandomForestRegressor(n_estimators = 500)
        model.fit(X, y, n_subsamples = n_sub, replace = False)
        pred.append(model.predict(test)[0])
#         var_c.append(predict_var(model, test)[0])
        var_uc.append(predict_var(model, test, method = "IJ")[0])

    target = np.mean(pred)
#     count_c = 0
#     for r in range(rep):
#         lower = pred[r] - 1.96 * np.sqrt(var_c[r])
#         upper = pred[r] + 1.96 * np.sqrt(var_c[r])
#         if target >= lower and target <= upper:
#             count_c += 1
    count_uc = 0
    for r in range(rep):
        lower = pred[r] - 1.96 * np.sqrt(var_uc[r])
        upper = pred[r] + 1.96 * np.sqrt(var_uc[r])
        if target >= lower and target <= upper:
            count_uc += 1        
    
    result_500[500]["normal"].append(stats.normaltest(pred))
#     result_100[500]["vr_c"].append(np.mean(var_c) / np.var(pred))
    result_500[500]["vr_uc"].append(np.mean(var_uc) / np.var(pred))
#     result_100[500]["cc_c"].append(count_c / rep)
    result_500[500]["cc_uc"].append(count_uc / rep)


    # print("========================= B = 1000 =========================")
    
    pred = []
    var_uc = []
    var_c = []

    for r in range(rep):

        data = df_train[incre * r: (incre * r + incre)]

        X = data[["F1", "F2", "F3", "F4", "F5", "F6", "F7", "F8", "F9"]].values
        y = data[["RMSD"]].values.ravel()

        model = RandomForestRegressor(n_estimators = 1000)
        model.fit(X, y, n_subsamples = n_sub, replace = False)
        pred.append(model.predict(test)[0])
#         var_c.append(predict_var(model, test)[0])
        var_uc.append(predict_var(model, test, method = "IJ")[0])

    target = np.mean(pred)
#     count_c = 0
#     for r in range(rep):
#         lower = pred[r] - 1.96 * np.sqrt(var_c[r])
#         upper = pred[r] + 1.96 * np.sqrt(var_c[r])
#         if target >= lower and target <= upper:
#             count_c += 1
    count_uc = 0
    for r in range(rep):
        lower = pred[r] - 1.96 * np.sqrt(var_uc[r])
        upper = pred[r] + 1.96 * np.sqrt(var_uc[r])
        if target >= lower and target <= upper:
            count_uc += 1        
    
    result_500[1000]["normal"].append(stats.normaltest(pred))
#     result_100[500]["vr_c"].append(np.mean(var_c) / np.var(pred))
    result_500[1000]["vr_uc"].append(np.mean(var_uc) / np.var(pred))
#     result_100[500]["cc_c"].append(count_c / rep)
    result_500[1000]["cc_uc"].append(count_uc / rep)



    # print("========================= B = 2500 =========================")
    pred = []
    var_uc = []
    var_c = []

    for r in range(rep):

        data = df_train[incre * r: (incre * r + incre)]

        X = data[["F1", "F2", "F3", "F4", "F5", "F6", "F7", "F8", "F9"]].values
        y = data[["RMSD"]].values.ravel()

        model = RandomForestRegressor(n_estimators = 2500)
        model.fit(X, y, n_subsamples = n_sub, replace = False)
        pred.append(model.predict(test)[0])
#         var_c.append(predict_var(model, test)[0])
        var_uc.append(predict_var(model, test, method = "IJ")[0])

    target = np.mean(pred)
#     count_c = 0
#     for r in range(rep):
#         lower = pred[r] - 1.96 * np.sqrt(var_c[r])
#         upper = pred[r] + 1.96 * np.sqrt(var_c[r])
#         if target >= lower and target <= upper:
#             count_c += 1
    count_uc = 0
    for r in range(rep):
        lower = pred[r] - 1.96 * np.sqrt(var_uc[r])
        upper = pred[r] + 1.96 * np.sqrt(var_uc[r])
        if target >= lower and target <= upper:
            count_uc += 1        
    
    result_500[2500]["normal"].append(stats.normaltest(pred))
#     result_100[500]["vr_c"].append(np.mean(var_c) / np.var(pred))
    result_500[2500]["vr_uc"].append(np.mean(var_uc) / np.var(pred))
#     result_100[500]["cc_c"].append(count_c / rep)
    result_500[2500]["cc_uc"].append(count_uc / rep)


    # print("========================= B = 5000 =========================")
    pred = []
    var_uc = []
    var_c = []

    for r in range(rep):

        data = df_train[incre * r: (incre * r + incre)]

        X = data[["F1", "F2", "F3", "F4", "F5", "F6", "F7", "F8", "F9"]].values
        y = data[["RMSD"]].values.ravel()

        model = RandomForestRegressor(n_estimators = 5000)
        model.fit(X, y, n_subsamples = n_sub, replace = False)
        pred.append(model.predict(test)[0])
#         var_c.append(predict_var(model, test)[0])
        var_uc.append(predict_var(model, test, method = "IJ")[0])

    target = np.mean(pred)
#     count_c = 0
#     for r in range(rep):
#         lower = pred[r] - 1.96 * np.sqrt(var_c[r])
#         upper = pred[r] + 1.96 * np.sqrt(var_c[r])
#         if target >= lower and target <= upper:
#             count_c += 1
    count_uc = 0
    for r in range(rep):
        lower = pred[r] - 1.96 * np.sqrt(var_uc[r])
        upper = pred[r] + 1.96 * np.sqrt(var_uc[r])
        if target >= lower and target <= upper:
            count_uc += 1        
    
    result_500[5000]["normal"].append(stats.normaltest(pred))
#     result_100[500]["vr_c"].append(np.mean(var_c) / np.var(pred))
    result_500[5000]["vr_uc"].append(np.mean(var_uc) / np.var(pred))
#     result_100[500]["cc_c"].append(count_c / rep)
    result_500[5000]["cc_uc"].append(count_uc / rep)



100%|██████████| 20/20 [8:37:54<00:00, 1553.73s/it]  


In [17]:
result_500

{500: {'normal': [NormaltestResult(statistic=2.8451091016641, pvalue=0.24109733416310816),
   NormaltestResult(statistic=3.5496887247886617, pvalue=0.16950982955056146),
   NormaltestResult(statistic=3.3249264890742753, pvalue=0.18967119766527912),
   NormaltestResult(statistic=2.4699672497304617, pvalue=0.2908395248652741),
   NormaltestResult(statistic=5.30473336330988, pvalue=0.07048420183973134),
   NormaltestResult(statistic=1.732669220212899, pvalue=0.42048998778825764),
   NormaltestResult(statistic=0.9687680076970377, pvalue=0.6160765807054969),
   NormaltestResult(statistic=0.4174066159439595, pvalue=0.8116360058569357),
   NormaltestResult(statistic=5.8146487553430575, pvalue=0.05462168151705984),
   NormaltestResult(statistic=7.968274671149231, pvalue=0.018608490283180484),
   NormaltestResult(statistic=0.43848967105697256, pvalue=0.8031250605315867),
   NormaltestResult(statistic=1.479962378748829, pvalue=0.47712289041667233),
   NormaltestResult(statistic=4.730801682495811

In [18]:
result = result_500

print("---------- B_n = 500 ----------")
B_n = 500
print("normality rejection rate: " + str(np.sum(np.array([i.pvalue for i in result[B_n]["normal"]]) < 0.05) / 20))
# print("mean variance ratio (corrected): " + str(np.mean(result[B_n]["vr_c"])))
# print("std variance ratio (corrected): " + str(np.std(result[B_n]["vr_c"])))
print("mean variance ratio (uncorrected): " + str(np.mean(result[B_n]["vr_uc"])))
print("std variance ratio (uncorrected): " + str(np.std(result[B_n]["vr_uc"])))
# print("mean coverage prob (corrected): " + str(np.mean(result[B_n]["cc_c"])))
# print("std coverage prob (corrected): " + str(np.std(result[B_n]["cc_c"])))
print("mean coverage prob (uncorrected): " + str(np.mean(result[B_n]["cc_uc"])))
print("std coverage prob (uncorrected): " + str(np.std(result[B_n]["cc_uc"])))

print("")
print("---------- B_n = 1000 ----------")
B_n = 1000
print("normality rejection rate: " + str(np.sum(np.array([i.pvalue for i in result[B_n]["normal"]]) < 0.05) / 20))
# print("mean variance ratio (corrected): " + str(np.mean(result[B_n]["vr_c"])))
# print("std variance ratio (corrected): " + str(np.std(result[B_n]["vr_c"])))
print("mean variance ratio (uncorrected): " + str(np.mean(result[B_n]["vr_uc"])))
print("std variance ratio (uncorrected): " + str(np.std(result[B_n]["vr_uc"])))
# print("mean coverage prob (corrected): " + str(np.mean(result[B_n]["cc_c"])))
# print("std coverage prob (corrected): " + str(np.std(result[B_n]["cc_c"])))
print("mean coverage prob (uncorrected): " + str(np.mean(result[B_n]["cc_uc"])))
print("std coverage prob (uncorrected): " + str(np.std(result[B_n]["cc_uc"])))

print("")
print("---------- B_n = 2500 ----------")
B_n = 2500
print("normality rejection rate: " + str(np.sum(np.array([i.pvalue for i in result[B_n]["normal"]]) < 0.05) / 20))
# print("mean variance ratio (corrected): " + str(np.mean(result[B_n]["vr_c"])))
# print("std variance ratio (corrected): " + str(np.std(result[B_n]["vr_c"])))
print("mean variance ratio (uncorrected): " + str(np.mean(result[B_n]["vr_uc"])))
print("std variance ratio (uncorrected): " + str(np.std(result[B_n]["vr_uc"])))
# print("mean coverage prob (corrected): " + str(np.mean(result[B_n]["cc_c"])))
# print("std coverage prob (corrected): " + str(np.std(result[B_n]["cc_c"])))
print("mean coverage prob (uncorrected): " + str(np.mean(result[B_n]["cc_uc"])))
print("std coverage prob (uncorrected): " + str(np.std(result[B_n]["cc_uc"])))

print("")
print("---------- B_n = 5000 ----------")
B_n = 5000
print("normality rejection rate: " + str(np.sum(np.array([i.pvalue for i in result[B_n]["normal"]]) < 0.05) / 20))
# print("mean variance ratio (corrected): " + str(np.mean(result[B_n]["vr_c"])))
# print("std variance ratio (corrected): " + str(np.std(result[B_n]["vr_c"])))
print("mean variance ratio (uncorrected): " + str(np.mean(result[B_n]["vr_uc"])))
print("std variance ratio (uncorrected): " + str(np.std(result[B_n]["vr_uc"])))
# print("mean coverage prob (corrected): " + str(np.mean(result[B_n]["cc_c"])))
# print("std coverage prob (corrected): " + str(np.std(result[B_n]["cc_c"])))
print("mean coverage prob (uncorrected): " + str(np.mean(result[B_n]["cc_uc"])))
print("std coverage prob (uncorrected): " + str(np.std(result[B_n]["cc_uc"])))

---------- B_n = 500 ----------
normality rejection rate: 0.1
mean variance ratio (uncorrected): 6.888075140464517
std variance ratio (uncorrected): 1.7314209228740336
mean coverage prob (uncorrected): 0.9988888888888889
std coverage prob (uncorrected): 0.004843221048378534

---------- B_n = 1000 ----------
normality rejection rate: 0.1
mean variance ratio (uncorrected): 3.622309850365043
std variance ratio (uncorrected): 1.0123657139797673
mean coverage prob (uncorrected): 0.9933333333333334
std coverage prob (uncorrected): 0.014229164972072987

---------- B_n = 2500 ----------
normality rejection rate: 0.15
mean variance ratio (uncorrected): 1.667617944291464
std variance ratio (uncorrected): 0.4026406935274372
mean coverage prob (uncorrected): 0.9677777777777777
std coverage prob (uncorrected): 0.031011746082117465

---------- B_n = 5000 ----------
normality rejection rate: 0.15
mean variance ratio (uncorrected): 1.0117564684042146
std variance ratio (uncorrected): 0.240429469828546

In [30]:
X = df[["F1", "F2", "F3", "F4", "F5", "F6", "F7", "F8", "F9"]].values
y = df[["RMSD"]].values.ravel()

In [31]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

In [47]:
regr = RandomForestRegressor()
regr.fit(X_train[:3000], y_train[:3000])

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [48]:
np.mean((regr.predict(X_test) - y_test) ** 2)

21.76038640147168

##### 